In [7]:
import numpy as np
import pandas as pd
import random
import re

In [8]:
"""
函数说明:接收一个大字符串并将其解析为字符串列表
Parameters:
    无
Returns:
    无
"""
def textParse(bigString):                                                   #将字符串转换为字符列表
    # * 会匹配0个或多个规则，split会将字符串分割成单个字符【python3.5+】; 这里使用\W 或者\W+ 都可以将字符数字串分割开，
    #产生的空字符将会在后面的列表推导式中过滤掉
    listOfTokens = re.split(r'\W+', bigString)                              #将特殊符号作为切分标志进行字符串切分，即非字母、非数字
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]            #除了单个字母，例如大写的I，其它单词变成小写

In [9]:
def get_dataSet():
    docList = []; classList = []
    for i in range(1, 26):                                                  #遍历25个txt文件
        wordList = textParse(open('email/spam/%d.txt' % i, 'r').read())     #读取每个垃圾邮件，并字符串转换成字符串列表
        docList.append(wordList)
        classList.append(1)                                                 #标记垃圾邮件，1表示垃圾文件
        wordList = textParse(open('email/ham/%d.txt' % i, 'r').read())      #读取每个非垃圾邮件，并字符串转换成字符串列表
        docList.append(wordList)
        classList.append(0)                 
    return docList,classList

In [22]:
docList,classList = get_dataSet()

In [23]:
classList

[1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0]

In [57]:
## 2、构建词汇表

In [60]:
"""
函数功能：将切分的样本词条整理成词汇表（不重复）
参数说明：
dataSet：切分好的样本词条
返回：
vocabList：不重复的词汇表
"""
def createVocablist(dataSet):
    vocabSet = set()               #创建一个空的集合，set可用于去重
    for doc in dataSet:
        vocabSet = vocabSet | set(doc)     #取并集
        vocablist = list(vocabSet)
    return vocablist   

In [61]:
##  生成词向量

In [63]:
"""
函数功能：根据vocabList词汇表，将inputSet向量化，向量的每个元素为1或0
参数说明：
vocabList：词汇表
inputSet：切分好的词条列表中的一条  （dataSet中的一条数据）
返回：
returnVec：文档向量,词集模型
"""
def setOfWords2Vec(vocablist, inputSet):
    returnVec = [0] * len(vocablist)                  #创建一个其中所含元素都为0的向量
    for word in inputSet:                             #遍历每个词条
        if word in vocablist:                         #如果词条存在于词汇表中，则变为1
            returnVec[vocablist.index(word)] = 1
        else:
            print(" {} is not in my Vocabulary!".format(word) )
    return returnVec                                  #返回文档向量

In [64]:
### 生成训练集向量列表

In [68]:
"""
函数功能：生成训练集向量列表
参数说明：
dataSet：切分好的样本词条
返回：
trainMat：所有的词条向量组成的列表
"""
def get_trainMat(dataSet):
    trainMat = []                                         #初始化向量列表
    vocablist = createVocablist(dataSet)                  #生成词汇表
    for inputSet in dataSet:                              #遍历样本词条中的每一条样本
        returnVec=setOfWords2Vec(vocablist, inputSet)     #将当前词条向量化
        trainMat.append(returnVec)                        #追加到向量列表中
    return trainMat

In [71]:
"""
函数功能：朴素贝叶斯分类器训练函数
参数说明：
trainMat：训练文档矩阵
classVec：训练类别标签向量  classVec = [0,1,0,1,0,1]
返回：
p0V：非侮辱类的条件概率数组
p1V：侮辱类的条件概率数组
pAb：文档属于侮辱类的概率
"""
def trainNB(trainMat,classVec):
    n = len(trainMat)                     #计算训练的文档数目
    m = len(trainMat[0])                  #计算每篇文档的词条数
    pAb = sum(classVec)/n                 #文档属于侮辱类的概率
    p0Num = np.ones(m)                    #词条出现数初始化为1
    p1Num = np.ones(m)                    #词条出现数初始化为1
    p0Denom = 2                           #分母初始化为2
    p1Denom = 2                           #分母初始化为2
    for i in range(n):                    #遍历每一个文档
        if classVec[i] == 1:              #统计属于侮辱类的条件概率所需的数据
            p1Num += trainMat[i]
            p1Denom += sum(trainMat[i])
        else:                             #统计属于非侮辱类的条件概率所需的数据
            p0Num += trainMat[i]
            p0Denom += sum(trainMat[i])
    p1V = np.log(p1Num/p1Denom)                                      
    p0V = np.log(p0Num/p0Denom)        
    return p0V,p1V,pAb                    #返回属于非侮辱类,侮辱类和文档属于侮辱类的概率

In [72]:
from functools import reduce
"""
函数功能：朴素贝叶斯分类器分类函数
参数说明：
vec2Classify：待分类的词条数组,已经向量化的测试样本
p0V：非侮辱类的条件概率数组
p1V：侮辱类的条件概率数组
pAb：文档属于侮辱类的概率
返回：
0：属于非侮辱类
1：属于侮辱类
"""
def classifyNB(vec2Classify, p0V, p1V, pAb):
    p1 = sum(vec2Classify * p1V) + np.log(pAb)       #对应元素相乘
    p0 = sum(vec2Classify * p0V) + np.log(1- pAb)    #对应元素相乘
    if p1 > p0:
        return 1
    else:
        return 0  

In [78]:
"""
函数功能：产生训练集和测试集的索引值

"""
def createtest():
    trainingSet = list(range(len(classList))); testSet = []                 #创建存储训练集的索引值的列表和测试集的索引值的列表  
    for i in range(10):                                                     #从50个邮件中，随机挑选出40个作为训练集,10个做测试集
        randIndex = int(random.uniform(0, len(trainingSet)))                #随机选取索索引值
        testSet.append(trainingSet[randIndex])                              #添加测试集的索引值
        del(trainingSet[randIndex])                                         #在训练集列表中删除添加到测试集的索引值   
    return trainingSet,testSet

In [79]:
trainingSet,testSet = createtest()

In [80]:
for docIndex in trainingSet:
    print(docList[docIndex])

['codeine', '15mg', 'for', '203', 'visa', 'only', 'codeine', 'methylmorphine', 'narcotic', 'opioid', 'pain', 'reliever', 'have', '15mg', '30mg', 'pills', '15mg', 'for', '203', '15mg', 'for', '385', '15mg', 'for', '562', 'visa', 'only']
['peter', 'with', 'jose', 'out', 'town', 'you', 'want', 'meet', 'once', 'while', 'keep', 'things', 'going', 'and', 'some', 'interesting', 'stuff', 'let', 'know', 'eugene']
['hydrocodone', 'vicodin', 'brand', 'watson', 'vicodin', '750', '195', '120', '570', 'brand', 'watson', '750', '195', '120', '570', 'brand', 'watson', '325', '199', '120', '588', 'noprescription', 'required', 'free', 'express', 'fedex', 'days', 'delivery', 'for', 'over', '200', 'order', 'major', 'credit', 'cards', 'check']
['yay', 'you', 'both', 'doing', 'fine', 'working', 'mba', 'design', 'strategy', 'cca', 'top', 'art', 'school', 'new', 'program', 'focusing', 'more', 'right', 'brained', 'creative', 'and', 'strategic', 'approach', 'management', 'the', 'way', 'done', 'today']
['you', '

In [81]:
"""
函数功能：朴素贝叶斯测试函数
参数说明：
testVec：测试样本
返回：测试样本的类别
"""
def testingNB():
    docList,classList = get_dataSet()                       #创建实验样本
    vocabList = createVocablist(docList)                                    #创建词汇表，不重复 
    trainMat = []; trainClasses = []                                        #创建训练集矩阵和训练集类别标签系向量             
    for docIndex in trainingSet:                                            #遍历训练集
        trainMat.append(setOfWords2Vec(vocabList, docList[docIndex]))       #将生成的词集模型添加到训练矩阵中
        trainClasses.append(classList[docIndex])                            #将类别添加到训练集类别标签系向量中
    p0V,p1V,pAb = trainNB(np.array(trainMat), np.array(trainClasses))               #训练朴素贝叶斯分类器
    errorCount = 0                                                          #错误分类计数
    for docIndex in testSet:                                                #遍历测试集
        wordVector = setOfWords2Vec(vocabList, docList[docIndex])           #测试集的词集模型
        if classifyNB(np.array(wordVector), p0V,p1V,pAb) != classList[docIndex]:    #如果分类错误
            errorCount += 1                                                 #错误计数加1
            print("分类错误的测试集：",docList[docIndex])
            print(docIndex)
    print('错误率：%.2f%%' % (float(errorCount) / len(testSet) * 100))

In [82]:
testingNB()

分类错误的测试集： ['oem', 'adobe', 'microsoft', 'softwares', 'fast', 'order', 'and', 'download', 'microsoft', 'office', 'professional', 'plus', '2007', '2010', '129', 'microsoft', 'windows', 'ultimate', '119', 'adobe', 'photoshop', 'cs5', 'extended', 'adobe', 'acrobat', 'pro', 'extended', 'windows', 'professional', 'thousand', 'more', 'titles']
10
错误率：10.00%


10